In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,37281
2,Huelva,Confirmados PDIA 14 días,2638
3,Huelva,Tasa PDIA 14 días,"514,059668336029"
4,Huelva,Confirmados PDIA 7 días,1344
5,Huelva,Tasa PDIA 7 dias,"261,9015141181285"
6,Huelva,Total Confirmados,37485
7,Huelva,Curados,33235
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  37281.0


/tmp/ipykernel_20237/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10399.0


/tmp/ipykernel_20237/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_20237/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_20237/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_20237/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 345 personas en los últimos 7 días 

Un positivo PCR cada 158 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,37281.0,1344.0,2638.0,513170.0,261.901514,514.059668,638.0
Huelva-Costa,21895.0,883.0,1797.0,289548.0,304.958073,620.622487,410.0
Huelva (capital),10399.0,417.0,909.0,143837.0,289.911497,631.965350,192.0
Condado-Campiña,11746.0,286.0,568.0,156231.0,183.062260,363.564209,149.0
Sierra de Huelva-Andévalo Central,3259.0,167.0,262.0,67391.0,247.807571,388.775949,71.0
Ayamonte,1582.0,77.0,150.0,21104.0,364.859742,710.765732,43.0
Isla Cristina,2546.0,95.0,183.0,21393.0,444.070490,855.419997,38.0
Moguer,1589.0,55.0,124.0,21867.0,251.520556,567.064526,35.0
Lepe,2444.0,89.0,161.0,27880.0,319.225251,577.474892,34.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,137.0,19.0,19.0,1310.0,1450.381679,1450.381679,16.0
Cortelazor,17.0,2.0,4.0,299.0,668.896321,1337.792642,0.0
Beas,281.0,27.0,46.0,4341.0,621.976503,1059.663672,12.0
Santa Ana la Real,16.0,5.0,5.0,475.0,1052.631579,1052.631579,2.0
Castaño del Robledo,9.0,2.0,2.0,228.0,877.192982,877.192982,0.0
Isla Cristina,2546.0,95.0,183.0,21393.0,444.070490,855.419997,38.0
Palos de la Frontera,856.0,46.0,100.0,11742.0,391.756089,851.643672,23.0
Punta Umbría,994.0,65.0,115.0,15355.0,423.314881,748.941713,33.0
Ayamonte,1582.0,77.0,150.0,21104.0,364.859742,710.765732,43.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,115.0,1.0,4.0,2118.0,47.214353,188.857413,1.0,0.250000
Calañas,213.0,3.0,11.0,2768.0,108.381503,397.398844,3.0,0.272727
Niebla,315.0,5.0,18.0,4116.0,121.477162,437.317784,1.0,0.277778
San Bartolomé de la Torre,320.0,2.0,7.0,3761.0,53.177346,186.120713,0.0,0.285714
Alosno,210.0,1.0,3.0,3933.0,25.425884,76.277651,0.0,0.333333
Santa Bárbara de Casa,38.0,1.0,3.0,1043.0,95.877277,287.631831,0.0,0.333333
Bonares,350.0,12.0,31.0,6060.0,198.019802,511.551155,2.0,0.387097
Almonaster la Real,75.0,2.0,5.0,1788.0,111.856823,279.642058,2.0,0.400000
Jabugo,75.0,3.0,7.0,2260.0,132.743363,309.734513,1.0,0.428571
